# Benchmarking de modèles pour la reconnaissance de tumeurs cérébrales 🧠

## Objectif

Dans ce notebook, nous allons comparer différents modèles de reconnaissance de tumeurs cérébrales à l'aide des données de test. Les objectifs sont :

1. **Évaluer les modèles** avec des matrices de confusion et des courbes ROC.
2. **Définir et utiliser des métriques agrégées** (e.g., precision, recall et $F_1$-score globaux) pour résumer les performances de chaque modèle sur toutes les classes.
3. **Analyser l'effet de l'équilibrage des classes** sur les performances des modèles.
4. **Comparer les performances globales** pour identifier le modèle le plus adapté.

---

## Méthodologie

### 1. Calcul des métriques pour les 4 classes

#### **Precision** :
La **precision** mesure la proportion des prédictions positives correctes pour une classe donnée par rapport à toutes les prédictions positives faites pour cette classe :

$$
\text{Precision}_i = \frac{\text{VP}_i}{\text{VP}_i + \text{FP}_i}
$$

- $\text{VP}_i$ (Vrais Positifs) : Nombre d'échantillons de la classe $i$ correctement prédits.
- $\text{FP}_i$ (Faux Positifs) : Nombre d'échantillons appartenant à d'autres classes mais incorrectement prédits comme appartenant à $i$.

---

#### **Recall** :
Le **recall** mesure la proportion des échantillons réellement appartenant à une classe qui ont été correctement identifiés :

$$
\text{Recall}_i = \frac{\text{VP}_i}{\text{VP}_i + \text{FN}_i}
$$

- $\text{FN}_i$ (Faux Négatifs) : Nombre d'échantillons de la classe $i$ mal prédits comme appartenant à d'autres classes.

---

#### **$F_1$-score** :
Le **$F_1$-score** combine precision et recall dans une moyenne harmonique. Il équilibre les deux métriques, particulièrement utile en cas de données déséquilibrées :

$$
F_{1,i} = 2 \cdot \frac{\text{Precision}_i \cdot \text{Recall}_i}{\text{Precision}_i + \text{Recall}_i}
$$

---

### 2. Métrique globale pour les 4 classes

Pour comparer les modèles de manière globale, nous définissons une **métrique agrégée** qui combine les scores de precision, recall et $F_1$ pour toutes les classes. Cela est réalisé via une **moyenne pondérée**, tenant compte du nombre d'échantillons dans chaque classe.

#### Formules des métriques globales :

- **Precision globale** (moyenne pondérée) :
$$
\text{Precision Global} = \frac{\sum_{i=1}^{4} n_i \cdot \text{Precision}_i}{\sum_{i=1}^{4} n_i}
$$

- **Recall global** (moyenne pondérée) :
$$
\text{Recall Global} = \frac{\sum_{i=1}^{4} n_i \cdot \text{Recall}_i}{\sum_{i=1}^{4} n_i}
$$

- **$F_1$-score global** (basé sur les totals agrégés pour toutes les classes) :
$$
F_1\text{-score Global} = 2 \cdot \frac{\text{Precision Global} \cdot \text{Recall Global}}{\text{Precision Global} + \text{Recall Global}}
$$

où :
- $n_i$ : Nombre d'échantillons dans la classe $i$.
- $\text{Precision}_i$, $\text{Recall}_i$ : Metrics pour la classe $i$.

---

### 3. Analyse et comparaison des modèles

1. **Évaluation des modèles individuellement pour chaque classe** :
   - Calculer precision, recall et $F_1$-score pour chaque classe (Classe 0, Classe 1, Classe 2, Classe 3).
   - Générer les matrices de confusion pour visualiser les erreurs de prédiction par classe.

2. **Calcul des métriques globales** :
   - Moyenne pondérée des metrics pour les 4 classes.
   - Comparer les valeurs de precision globale, recall global et $F_1$-score global entre les modèles.

3. **Visualisation des performances** :
   - Tracer des courbes ROC pour chaque classe.
   - Présenter les scores globaux sous forme de tableaux ou de diagrammes.

---

### 4. Impact de l'équilibrage des classes

Tester différentes approches pour gérer les déséquilibres dans les données, telles que :
- **Sur-échantillonnage** des classes minoritaires.
- **Sous-échantillonnage** des classes majoritaires.
- **Pondération des classes** dans les fonctions de coût.

Analyser l'effet de ces approches sur les metrics globales et individuelles.


---

## Conclusion

À la fin de ce notebook, nous serons en mesure de :
1. Identifier le modèle offrant les meilleures performances globales.
2. Comprendre comment chaque modèle gère les déséquilibres entre les classes.
3. Fournir des recommandations pour l'utilisation des modèles dans des scénarios réels.

In [1]:
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_curve, auc
)
from sklearn.preprocessing import label_binarize

from module_for_preprocessing_metrics import *  

with open('../config.yml', 'r') as file:
    config = yaml.safe_load(file)


size = config['données']['image']['size']


## Accès aux données de test et aux historiques des modèles 🔍

Dans cette section, nous allons charger et explorer nos données de test. Ces données seront utilisées pour évaluer les performances des modèles entraînés.

De plus, nous accédons également aux historiques des modèles (*history*), qui contiennent des informations sur l'évolution des métriques telles que la précision (*accuracy*) et la perte (*loss*) pendant l'entraînement. Ces historiques seront utilisés pour visualiser les performances des modèles au fil des époques.


In [4]:
testing = os.path.join(os.path.abspath(os.path.join(os.getcwd(), "..")), "data", "Testing")
x_test, y_test = load_images_with_preprocessing(testing, size)

models_history = os.path.join(os.path.join("..", "src", "models", "sauvegardes_modeles"), 'models_history.json')

# Glossaire des modèles utilisés dans le benchmark 🧠

Ce glossaire décrit les modèles évalués dans le cadre de notre benchmark. Chaque modèle présente une architecture ou une stratégie d'entraînement différente pour comparer leurs performances sur la reconnaissance de tumeurs cérébrales.

---

## **Baseline** 🤖
- **Description** : Modèle préentraîné basé sur **Inception** et optimisé pour le domaine médical.
- **Objectif** : Fournir un point de référence pour comparer les modèles personnalisés.  
📒 *Apparaît dans un notebook dedié.*

---

## **Model 1** 🤖
- **Architecture** :
  - 4 couches de convolution avec activation **ReLU**.
  - 2 couches de **max pooling** pour la réduction dimensionnelle.
  - 1 couche fully connected avec 64 neurones et activation **ReLU**.
  - 1 couche de **Dropout** (20% de neurones désactivés pour éviter le surapprentissage).
  - 1 couche de sortie avec activation **softmax** pour classer en 4 catégories.
- **Particularité** : 
  - 🚫 **Sans augmentation de données** : Les données d'entraînement sont utilisées telles quelles, sans modification ni équilibrage.

---

## **Model 2** 🤖
- **Architecture** :
  - Identique au **Model 1** :
    - 4 couches de convolution avec activation **ReLU**.
    - 2 couches de **max pooling**.
    - 1 couche fully connected avec 64 neurones et activation **ReLU**.
    - 1 couche de **Dropout** (20%).
    - 1 couche de sortie avec activation **softmax**.
- **Particularité** :
  - ✅ **Avec augmentation de données** :
    - Équilibrage des classes : Ajout d'images synthétiques pour égaliser le nombre d'images dans chaque classe.
    - Transformations appliquées : **Zoom**, **décalage** et autres opérations pour enrichir les données et améliorer la robustesse du modèle.

---

## **Model 3** 🤖
- **Architecture** :
  - 4 couches de convolution avec activation **sigmoid**.
  - 2 couches de **max pooling**.
  - 1 couche fully connected avec 64 neurones et activation **ReLU**.
  - 1 couche de **Dropout** (20%).
  - 1 couche de sortie avec activation **softmax** pour classer en 4 catégories.
- **Particularité** :
  - ✨ Utilisation de l'activation **sigmoid** dans les couches de convolution, contrairement aux modèles précédents qui utilisent **ReLU**.

---

## **Résumé des différences** 📊
| Modèle       | Architecture                                                                                      | Augmentation de données                 | Activation des convolutions |
|--------------|--------------------------------------------------------------------------------------------------|-----------------------------------------|-----------------------------|
| **Baseline** | Modèle préentraîné Inception optimisé pour le domaine médical                                     | Non                                     | N/A                         |
| **Model 1**  | Sequential avec 4 convolutions (ReLU), 2 max pooling, 1 fully connected (ReLU), Dropout (20%)     | Non                                     | **ReLU**                    |
| **Model 2**  | Sequential identique au Model 1                                                                  | Oui (équilibrage, zoom, décalage)       | **ReLU**                    |
| **Model 3**  | Sequential avec 4 convolutions (sigmoid), 2 max pooling, 1 fully connected (ReLU), Dropout (20%) | Non                                     | **Sigmoid**                 |


In [2]:
#baseline = load_saved_model('inception1.h5')
model_1 = load_saved_model('modele_brain_tumor_20241117_205951.h5')
model_2 = load_saved_model('modele_brain_tumor_20241125_002219.h5')
model_3 = load_saved_model('modele_brain_tumor_20241210_123938.h5')

# Analyse du Modèle 1

In [8]:
plot_confusion_matrix(model_1,x_test,y_test)

13/13 [==============================] - 43s 3s/step


Classification Report:
              precision    recall  f1-score   support

      Glioma       0.81      0.17      0.28       100
  Meningioma       0.68      0.88      0.77       115
    No Tumor       0.58      0.96      0.73       105
   Pituitary       0.88      0.61      0.72        74

    accuracy                           0.67       394
   macro avg       0.74      0.65      0.62       394
weighted avg       0.72      0.67      0.62       394



In [9]:
courbe_ROC(model_1,x_test,y_test)

13/13 [==============================] - 41s 3s/step


# Analyse du Modèle 1

## 1. Matrice de confusion

La matrice de confusion révèle les performances du modèle sur chaque classe :

- **Glioma** :
  - Vrais positifs (VP) : 17
  - Faux positifs (FP) : 1 + 1 + 2 = 4
  - Faux négatifs (FN) : 24 + 56 + 3 = 83

- **Meningioma** :
  - Vrais positifs (VP) : 101
  - Faux positifs (FP) : 24 + 3 + 21 = 48
  - Faux négatifs (FN) : 1 + 10 + 3 = 14

- **No Tumor** :
  - Vrais positifs (VP) : 101
  - Faux positifs (FP) : 56 + 10 + 6 = 72
  - Faux négatifs (FN) : 1 + 3 + 0 = 4

- **Pituitary** :
  - Vrais positifs (VP) : 45
  - Faux positifs (FP) : 3 + 3 + 0 = 6
  - Faux négatifs (FN) : 2 + 21 + 6 = 29

### Observations :
- Le modèle est performant pour **Meningioma**, **No Tumor**, et dans une moindre mesure pour **Pituitary**.
- La classe **Glioma** reste problématique avec un nombre important de faux négatifs.
- Les confusions entre **Pituitary** et les autres classes demeurent un point à améliorer.

---

## 2. Courbes ROC et AUC
Les courbes ROC et les scores AUC permettent de visualiser les performances globales du modèle pour chaque classe.

- **Glioma** : AUC = 0.57
  - La performance est faible, indiquant que le modèle a des difficultés à différencier cette classe des autres.

- **Meningioma** : AUC = 0.91
  - La performance est excellente, montrant une bonne différenciation entre cette classe et les autres.

- **No Tumor** : AUC = 0.94
  - Le modèle distingue très bien cette classe des autres.

- **Pituitary** : AUC = 0.94
  - La performance est excellente pour cette classe.

### Observations :
- Les classes **No Tumor** et **Pituitary** sont bien différenciées des autres, comme l'indiquent leurs scores AUC élevés.
- La classe **Glioma** montre des performances faibles, nécessitant une amélioration.
- Les scores AUC de **Meningioma**, **No Tumor**, et **Pituitary** montrent que le modèle est efficace pour ces classes, malgré quelques confusions.



# Analyse du Modèle 2

In [10]:
plot_confusion_matrix(model_2,x_test,y_test)

13/13 [==============================] - 45s 3s/step


Classification Report:
              precision    recall  f1-score   support

      Glioma       0.77      0.17      0.28       100
  Meningioma       0.69      0.84      0.76       115
    No Tumor       0.56      0.92      0.70       105
   Pituitary       0.88      0.69      0.77        74

    accuracy                           0.66       394
   macro avg       0.73      0.66      0.63       394
weighted avg       0.71      0.66      0.62       394



In [11]:
courbe_ROC(model_2,x_test,y_test)

13/13 [==============================] - 39s 3s/step


# Analyse du Modèle 2

## 1. Matrice de confusion

La matrice de confusion révèle les performances du modèle sur chaque classe :

- **Glioma** :
  - Vrais positifs (VP) : 17
  - Faux positifs (FP) : 1 + 3 + 1 = 5
  - Faux négatifs (FN) : 27 + 52 + 4 = 83

- **Meningioma** :
  - Vrais positifs (VP) : 97
  - Faux positifs (FP) : 27 + 4 + 12 = 43
  - Faux négatifs (FN) : 1 + 15 + 2 = 18

- **No Tumor** :
  - Vrais positifs (VP) : 97
  - Faux positifs (FP) : 52 + 15 + 10 = 77
  - Faux négatifs (FN) : 3 + 4 + 1 = 8

- **Pituitary** :
  - Vrais positifs (VP) : 51
  - Faux positifs (FP) : 4 + 2 + 1 = 7
  - Faux négatifs (FN) : 1 + 12 + 10 = 23

### Observations :
- Le modèle est performant pour **Meningioma**, **No Tumor**, et dans une moindre mesure pour **Pituitary**.
- La classe **Glioma** est mal détectée, avec un faible nombre de vrais positifs et de nombreux faux négatifs.
- Les confusions entre **Pituitary** et les autres classes demeurent un point à améliorer.

---

## 2. Courbes ROC et AUC
Les courbes ROC et les scores AUC permettent de visualiser les performances globales du modèle pour chaque classe.

- **Glioma** : AUC = 0.53
  - La performance est faible, indiquant que le modèle a des difficultés à différencier cette classe des autres.

- **Meningioma** : AUC = 0.89
  - La performance est bonne, montrant une différenciation correcte entre cette classe et les autres.

- **No Tumor** : AUC = 0.90
  - Le modèle distingue bien cette classe des autres.

- **Pituitary** : AUC = 0.93
  - La performance est excellente pour cette classe.

### Observations :
- Les classes **No Tumor** et **Pituitary** sont bien différenciées des autres, comme l'indiquent leurs scores AUC élevés.
- La classe **Glioma** montre des performances faibles, nécessitant une amélioration pour mieux capturer ses caractéristiques distinctives.
- Les scores AUC de **Meningioma**, **No Tumor**, et **Pituitary** montrent que le modèle est efficace pour ces classes, malgré quelques confusions.


# Analyse du Modèle 3

In [12]:
plot_confusion_matrix(model_3,x_test,y_test)

13/13 [==============================] - 40s 3s/step


Classification Report:
              precision    recall  f1-score   support

      Glioma       0.25      1.00      0.40       100
  Meningioma       0.00      0.00      0.00       115
    No Tumor       0.00      0.00      0.00       105
   Pituitary       0.00      0.00      0.00        74

    accuracy                           0.25       394
   macro avg       0.06      0.25      0.10       394
weighted avg       0.06      0.25      0.10       394



/Users/lilya/Downloads/M2 ISDS/Projects/Brain_tumor/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/lilya/Downloads/M2 ISDS/Projects/Brain_tumor/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/lilya/Downloads/M2 ISDS/Projects/Brain_tumor/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [13]:
courbe_ROC(model_3,x_test,y_test)

13/13 [==============================] - 47s 4s/step


# Analyse du Modèle 3

## 1. Matrice de confusion

La matrice de confusion révèle les performances du modèle sur chaque classe :

- **Glioma** :
  - Vrais positifs (VP) : 100
  - Faux positifs (FP) : 115 + 105 + 74 = 294
  - Faux négatifs (FN) : 0
  
- **Meningioma** :
  - Vrais positifs (VP) : 0
  - Faux positifs (FP) : 0
  - Faux négatifs (FN) : 115

- **No Tumor** :
  - Vrais positifs (VP) : 0
  - Faux positifs (FP) : 0
  - Faux négatifs (FN) : 105

- **Pituitary** :
  - Vrais positifs (VP) : 0
  - Faux positifs (FP) : 0
  - Faux négatifs (FN) : 74

### Observations :
- Le modèle prédit exclusivement la classe **Glioma**, ignorant complètement les autres classes.
- Cela indique un problème grave dans l'entraînement du modèle, tel qu'un fort déséquilibre des données ou une erreur dans la configuration de la sortie des prédictions.
Nous verrons plus tard l'effet de la fonction sigmoid qui est à l'origine de ce desequilibre.
---

## 2. Courbes ROC et AUC
Les courbes ROC et les scores AUC permettent de visualiser les performances globales du modèle pour chaque classe.

- **Glioma** : AUC = 0.57
  - La performance est faible, indiquant que le modèle a des difficultés à différencier cette classe des autres.

- **Meningioma** : AUC = 0.51
  - La performance est moyenne, montrant une différenciation limitée entre cette classe et les autres.

- **No Tumor** : AUC = 0.35
  - La performance est faible, le modèle ne distingue pas bien cette classe des autres.

- **Pituitary** : AUC = 0.23
  - La performance est très faible, montrant une incapacité à différencier cette classe des autres.

### Observations

Les résultats obtenus sont très mauvais, à tel point qu'une prédiction aléatoire serait plus efficace.  
Cela peut s'expliquer par une saturation de la fonction **sigmoid**, qui limite la capacité du modèle à apprendre des données de manière efficace.


# Équilibre des Classes : Data Augmentation avec ImageDataGenerator

## Qu'est-ce que l'ImageDataGenerator ?
L'`ImageDataGenerator` est une classe de la bibliothèque **Keras** utilisée pour générer de nouvelles images à partir des données existantes. Elle applique des transformations comme des rotations, des zooms ou des décalages afin d'augmenter artificiellement la taille et la diversité du jeu de données.

## Transformations Appliquées
Pour **équilibrer les classes et améliorer la robustesse du modèle**, nous avons utilisé les transformations suivantes :
 
- **Rotations** : Pour varier les orientations des images.  
- **Zooms** : Pour simuler des variations de taille ou d'échelle.  
- **Décalages horizontaux et verticaux** : Pour introduire des variations dans les positions.

## Objectifs
1. **Équilibrer les classes** : Générer plus d'exemples pour les classes sous-représentées.  
2. **Améliorer la robustesse du modèle** : Le rendre plus résilient aux variations des données du monde réel.

Ces techniques permettent d'augmenter la diversité des données et de réduire le surapprentissage, contribuant ainsi à des performances globales améliorées.


# Modele 1 VS Modele 2

In [14]:
plot_model_curves(models_history, 'modele_brain_tumor_20241117_205951')

In [15]:
plot_model_curves(models_history, 'modele_brain_tumor_20241125_002219')

## Analyse des performances : Loss et Accuracy

Les courbes de **loss** et d'**accuracy** semblent similaires pour les deux modèles étudiés. Afin d'approfondir notre comparaison, analysons leurs performances sur le jeu de test.


In [16]:
print('Test Accuracy du modele 1')
print('')
print(get_test_accuracy(models_history, 'modele_brain_tumor_20241117_205951'))
print('')
print('Test Accuracy du modele 2')
print('')
print(get_test_accuracy(models_history, 'modele_brain_tumor_20241125_002219'))

Test Accuracy du modele 1

0.6701

Test Accuracy du modele 2

0.6649746298789978


Nous observons que les **test accuracy** sont équivalents. Cela indique que l'équilibre des classes et l'augmentation des données n'apportent pas d'amélioration *significative*.



# Comparaison de Fonctions d'activation 

### Fonctions d'activation utilisées
- **ReLU** (Rectified Linear Unit) : pour introduire de la non-linéarité et éviter les problèmes de gradient nul.
- **Sigmoid** : pour normaliser les sorties entre 0 et 1, souvent utilisée pour des tâches de classification binaire.

# Modele 1 VS Modele 3

In [17]:
plot_model_curves(models_history, 'modele_brain_tumor_20241117_205951')

In [18]:
plot_model_curves(models_history, 'modele_brain_tumor_20241210_123938')

In [19]:
print('Test Accuracy du modele 1')
print('')
print(get_test_accuracy(models_history, 'modele_brain_tumor_20241117_205951'))
print('')
print('Test Accuracy du modele 3')
print('')
print(get_test_accuracy(models_history, 'modele_brain_tumor_20241210_123938'))

Test Accuracy du modele 1

0.6701

Test Accuracy du modele 3

0.2538


# Analyse des Performances des Modèles

Nous constatons que les **test accuracy** varient considérablement, avec le modèle 3 affichant des performances nettement inférieures. Une explication détaillée sera fournie dans le rapport pour analyser cette différence.

# Benchmarking Général 📊

En utilisant les métriques définies au début ✅


In [8]:
#--accuracy--
accuracy_1 = get_test_accuracy(models_history, 'modele_brain_tumor_20241117_205951')
accuracy_2 = get_test_accuracy(models_history, 'modele_brain_tumor_20241125_002219')
accuracy_3 = get_test_accuracy(models_history, 'modele_brain_tumor_20241210_123938')

#--loss--
loss_1 = get_test_loss(models_history, 'modele_brain_tumor_20241117_205951')
loss_2 = get_test_loss(models_history, 'modele_brain_tumor_20241125_002219')
loss_3 = get_test_loss(models_history, 'modele_brain_tumor_20241210_123938')


In [10]:
accuracy = [accuracy_1,accuracy_2,accuracy_3]

labels = [
    'Model 1',
    'Model 2',
    'Model 3'
]

plot_accuracy_histogram(accuracy, labels)

In [ ]:
model_1_metric = classification_report_aggregated(model_1, x_test, y_test)
model_2_metric = classification_report_aggregated(model_2, x_test, y_test)
model_3_metric = classification_report_aggregated(model_3, x_test, y_test)

13/13 [==============================] - 37s 3s/step


/Users/lilya/Downloads/M2 ISDS/Projects/Brain_tumor/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/lilya/Downloads/M2 ISDS/Projects/Brain_tumor/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/lilya/Downloads/M2 ISDS/Projects/Brain_tumor/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
df= pd.DataFrame()
df['Models'] = [f'model_{i}' for i in range(1,4)]
df['Precision'] = [model_1_metric['precision_avg'],model_2_metric['precision_avg'],model_3_metric['precision_avg']]
df['F1-score'] = [model_1_metric['f1_score_avg'],model_2_metric['f1_score_avg'],model_3_metric['f1_score_avg']]
df['Recall'] = [model_1_metric['recall_avg'],model_2_metric['recall_avg'],model_3_metric['recall_avg']]
df['Accuracy'] = [accuracy_1,accuracy_2,accuracy_3]
df['Loss'] = [loss_1,loss_2,loss_3]
df

,Models,Precision,F1-score,Recall,Accuracy,Loss
0,model_1,0.724620,0.623519,0.670051,0.670100,4.159100
1,model_2,0.712068,0.623227,0.664975,0.664975,4.787893
2,model_3,0.064418,0.102756,0.253807,0.253800,1.429900


In [ ]:

df_melted = df.melt(id_vars='Models', var_name='Metrics', value_name='Values')

fig = px.bar(
    df_melted,
    x='Metrics',
    y='Values',
    color='Models',
    barmode='group',
    title="Performance Metrics by Model",
    labels={"Values": "Scores"}
)

fig.show()

# Analyse des Performances des Modèles

Le graphique présente une comparaison des performances de trois modèles (`model_1`, `model_2`, `model_3`) à l’aide des métriques suivantes : **Précision (Precision)**, **F1-score**, **Rappel (Recall)**, **Exactitude (Accuracy)**, et **Perte (Loss)**.

## Interprétation des Résultats

1. **Précision, F1-score, Rappel et Exactitude** :
   - Les modèles `model_1` et `model_2` montrent des performances similaires et élevées sur ces métriques. Cela indique une bonne capacité à prédire correctement les classes cibles, bien qu'elles s soient pas excellente. 
   - Le modèle `model_3`, en revanche, affiche des scores nettement plus faibles, trés mauvais.

2. **Perte (Loss)** :
   - Le modèle `model_2` a une perte (loss) nettement plus élevée que les autres, ce qui pourrait indiquer un problème d’optimisation ou un surapprentissage (**overfitting** , peut-être dû au fait que l'augmentation de données n'a été faite que dans le but d'équilibrer les classes et donc de montrer de nombreuses fois la même image légèrement modifiée dans les classes sous-représentées.)
   - Bien que le modèle `model_3` ait la plus faible perte, ses faibles performances sur les autres métriques montrent qu’une faible perte ne garantit pas une bonne généralisation.

## Conclusion
- **Modèle recommandé** : Le modèle `model_1` est le plus équilibré, combinant de bonnes performances sur toutes les métriques. Il est donc préférable.
- **Points d’amélioration** :
  - Le modèle `model_2` nécessite une attention particulière pour résoudre son problème de perte élevée, l'augmentation de données n'étant pas plus efficace.
  - Le modèle `model_3` n'est pas meilleurs que de choisir au hasard une classe pour une image donnée.



Ce notebook est à mi-chemin entre la présentation des résultats et une étude de quelques paramètres.

## Limitations

En raison d'un manque de ressources, nous nous sommes limités à **6 epochs** et avons évité d'utiliser des architectures trop lourdes. 

## Perspectives

Dans le rapport final, nous détaillerons comment, idéalement, nous aurions calibré notre modèle pour obtenir une **meilleure accuracy**, selon nos recherches et analyses.
